In [1]:
import api_util

In [2]:
api_util.video_df

,video_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,favouriteCount,commentCount,duration,definition,caption
0,EuISxGGBflY,Danny Gonzalez,The Lying King,Thanks to SoFi for sponsoring the video! Enter...,"[danny gonzalez, funny, commentary, comedy, re...",2022-12-15T18:00:05Z,2564315,167646,None,7236,PT16M32S,hd,false
1,6sh2Ib3DklI,Danny Gonzalez,I Tried Walmart's Terrifying Metaverse Experience,Thanks to ExpressVPN for sponsoring this video...,"[danny gonzalez, funny, commentary, comedy, re...",2022-11-22T18:00:07Z,4030239,204562,None,11670,PT23M29S,hd,false
2,_tgGmYerbB4,Danny Gonzalez,Trying To Find The Worst iPhone Game 2,edited by Jake Mayer https://www.instagram.com...,"[danny gonzalez, funny, commentary, comedy, re...",2022-10-27T16:00:24Z,4415629,220768,None,9331,PT29M4S,hd,false
3,zCoMjByGJZw,Danny Gonzalez,I Tried Ghost Hunting To See If It's Fake,spooky scary stuff\n\nshot by https://www.inst...,"[danny gonzalez, funny, commentary, comedy, re...",2022-10-01T16:00:38Z,7415724,387341,None,17863,PT25M45S,hd,false
4,H2fHTJZqaKA,Danny Gonzalez,The Insane Story Of The Most Dangerous Movie E...,Thanks to ExpressVPN for sponsoring this video...,"[danny gonzalez, funny, commentary, comedy, re...",2022-09-24T17:00:07Z,3036463,179586,None,5676,PT19M23S,hd,false
...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,GD43C6CvdVQ,Danny Gonzalez,The 9 Friends Everyone Has - Danny Gonzalez,We all know these 9 types of people.\nFollow m...,"[vine, danny gonzalez, buzzfeed, funny list, l...",2014-12-03T00:55:12Z,247075,13963,None,775,PT2M7S,hd,false
266,qvMOO4-JSeg,Danny Gonzalez,Go Dumb - Danny Gonzalez,This is a video I made a while ago for an old ...,"[danny gonzalez, vine, comedy, hilarious, popu...",2014-10-10T20:45:37Z,277013,11995,None,427,PT56S,hd,false
267,WRe_yup6Nw0,Danny Gonzalez,The Dying Squid - Danny Gonzalez,This is a video I made a while ago for an old ...,"[dance, funny, song, hilarious, danny gonzalez...",2014-10-03T16:21:36Z,552288,18274,None,1264,PT2M13S,hd,false
268,wtiEuGdHAIo,Danny Gonzalez,Don't Do Your Homework - Danny Gonzalez,This is a video I made a while ago on a differ...,"[homework, funny, vine, danny gonzalez, music ...",2014-10-03T03:54:27Z,459777,18176,None,1496,PT50S,hd,false
